In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from load_pvoc_data import load_data, TRAIN_LENGTH

In [3]:
BATCH_SIZE = 32
EPOCHS = 4
VALIDATION_SPLIT = 0.3

In [4]:
def preprocessing(img, lbl):
    crop_img = tf.image.central_crop(img, 1)
    resized = tf.image.resize_images(img, (256, 256))
    norm_img = tf.image.per_image_standardization(resized)
    
    one_hot = tf.one_hot(lbl, 20)
    summed = tf.reduce_sum(one_hot, axis=-2)
    multi_hot = tf.where(
        tf.equal(summed, 0), tf.zeros_like(summed, dtype=tf.float32), tf.ones_like(summed, dtype=tf.float32)
    )
    return norm_img, multi_hot

In [5]:
def train_input_fn():
    train_dataset = tf.data.Dataset.from_generator(
        lambda:load_data("train"),
        (tf.uint8, tf.int32),
        (tf.TensorShape([None, None, 3]), tf.TensorShape([None]))
    ).map(preprocessing).shuffle(10000)
    train_dataset = train_dataset.apply(tf.contrib.data.assert_element_shape((
        [256, 256, 3],
        [20]
    )))
    
    val_length = int(VALIDATION_SPLIT * TRAIN_LENGTH)
    val_dataset = train_dataset.take(val_length).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))
    train_dataset = train_dataset.skip(val_length).apply(
        tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE)).repeat()

    return train_dataset, val_dataset

In [6]:
def test_input_fn():
    test_dataset = tf.data.Dataset.from_generator(
        lambda:load_data("test"),
        (tf.uint8, tf.int32),
        (tf.TensorShape([None, None, 3]), tf.TensorShape([None]))
    )
    test_dataset = test_dataset.map(preprocessing).apply(tf.contrib.data.assert_element_shape((
        [256, 256, 3],
        [20]
    )))
    return test_dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

In [7]:
def conv_layer(inputs, scope_name, eta=None, filters=32, kernel_size=3, strides=1):
    with tf.variable_scope(scope_name):
        w = tf.get_variable('conv_w', (kernel_size, kernel_size, int(inputs.shape[-1]), filters))
        b = tf.get_variable('conv_b', (filters,))
        if eta is not None:
            alpha = tf.get_variable('conv_alpha', (kernel_size, kernel_size, int(inputs.shape[-1]), filters))
            hebb = tf.get_variable('conv_hebb', (kernel_size, kernel_size, int(inputs.shape[-1]), filters),
                                   trainable=False, initializer=tf.zeros_initializer)
            w = w + tf.multiply(alpha, hebb)
    
    x = tf.nn.conv2d(input=inputs, filter=w, strides=[1, strides, strides, 1], padding='SAME') + b
    
    if eta is not None:
        # y is to be the output reshaped so as to be used as a kernel for convolution on input to get Hebbian update
        y = tf.image.resize_images(x, [int(inputs.shape[1])] * 2)
        y = tf.transpose(y, [1, 2, 0, 3])

        # in_mod is the input padded a/c to prev. convolution
        in_mod = tf.pad(inputs, [
            [0, 0],
            *([[int(np.floor((kernel_size - 1) / 2)), int(np.ceil((kernel_size - 1) / 2))]] * 2),
            [0, 0]
        ])
        # in_mod is now modded so as to preserve channels and sum over mini-batch samples for Hebbian update convolution
        in_mod = tf.transpose(in_mod, [3, 1, 2, 0])

        hebb_update = tf.nn.conv2d(input=in_mod, filter=y, strides=([1] * 4), padding='VALID')
        hebb = eta * tf.transpose(hebb_update, [1, 2, 0, 3]) + (1 - eta) * hebb
        
    return tf.nn.leaky_relu(tf.layers.batch_normalization(x))

In [8]:
def residual_block(inputs, filters, scope_name, eta=None):
    with tf.variable_scope(scope_name):
        x = conv_layer(inputs, scope_name='blk_layer_1', eta=eta, filters=filters, kernel_size=1)
        x = conv_layer(inputs, scope_name='blk_layer_2', eta=eta, filters=(filters * 2))
    return x + inputs

In [9]:
def darknet_block(inputs, filters, repetitions, scope_name, eta=None):
    with tf.variable_scope(scope_name):
        x = conv_layer(inputs, scope_name='blk_layer_0', eta=eta, filters=filters, strides=2)
        for i in range(repetitions):
            x = residual_block(x, filters / 2, scope_name='blk_rep_' + str(i), eta=eta)
    return x

In [10]:
def darknet_model(features, labels, mode):  
    features = tf.cast(features, dtype=tf.float32)
    
    eta = tf.get_variable('eta', (1,), initializer=tf.truncated_normal_initializer)
    
    x = conv_layer(features, scope_name='first', eta=eta, filters=32)
    x = darknet_block(x, filters=64, repetitions=1, scope_name='dark_blk_0', eta=eta)
    x = darknet_block(x, filters=128, repetitions=2, scope_name='dark_blk_1', eta=eta)
    x = darknet_block(x, filters=256, repetitions=8, scope_name='dark_blk_2', eta=eta)
    x = darknet_block(x, filters=512, repetitions=8, scope_name='dark_blk_3', eta=eta)
    x = darknet_block(x, filters=1024, repetitions=4, scope_name='dark_blk_4')
    
    x = tf.layers.average_pooling2d(x, pool_size=8, strides=8)
    x = tf.layers.flatten(x)
    x = tf.layers.dense(inputs=x, units=20, kernel_initializer=tf.truncated_normal_initializer())
    
    classes = tf.where(tf.sigmoid(x) >= 0.5, tf.ones_like(x, dtype=tf.float32), tf.zeros_like(x, dtype=tf.float32))
    correct_prediction = tf.equal(classes, labels)
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc = tf.identity(acc, name='accuracy_tensor')
    
    predictions = {'classes': classes, 'accuracy': acc}
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=x)
    loss = tf.reduce_mean(loss)
    
    tf.summary.scalar('accuracy', acc)
    tf.summary.scalar('loss', loss)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.contrib.estimator.TowerOptimizer(tf.train.AdamOptimizer(1e-4))
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=classes)}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss,
                                      eval_metric_ops=eval_metric_ops)


In [11]:
tensors_to_log = {'accuracy': 'accuracy_tensor'}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10)

In [12]:
model = tf.estimator.Estimator(
    model_fn=tf.contrib.estimator.replicate_model_fn(darknet_model),
    model_dir='/tmp/tmpupper', config=tf.estimator.RunConfig(
        save_checkpoints_steps=150, save_summary_steps=10, log_step_count_steps=10
    )
)

INFO:tensorflow:Replicating the `model_fn` across ['/device:GPU:0', '/device:GPU:1'].  Variables are going to be placed on ['/CPU:0'].  Consolidation device is going to be /CPU:0.
INFO:tensorflow:Using config: {'_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5e3318c8d0>, '_model_dir': '/tmp/tmpupper', '_log_step_count_steps': 10, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 150, '_service': None, '_keep_checkpoint_max': 5, '_session_config': None, '_save_checkpoints_secs': None, '_evaluation_master': '', '_is_chief': True, '_train_distribute': None, '_num_worker_replicas': 1, '_master': '', '_global_id_in_cluster': 0, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0}


In [13]:
validation_hook = tf.contrib.learn.monitors.replace_monitors_with_hooks(
    [tf.contrib.learn.monitors.ValidationMonitor(
        input_fn=lambda:train_input_fn()[1], every_n_steps=100, early_stopping_rounds=200
    )],
    model
)[0]

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


In [14]:
max_steps = int(((1 - VALIDATION_SPLIT) * TRAIN_LENGTH / BATCH_SIZE) * EPOCHS)
model.train(input_fn=lambda:train_input_fn()[0], hooks=[logging_hook, validation_hook],
            max_steps=max_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpupper/model.ckpt.
INFO:tensorflow:accuracy = 0.421875
INFO:tensorflow:step = 0, loss = 167.94116
INFO:tensorflow:global_step/sec: 1.67602
INFO:tensorflow:accuracy = 0.921875 (5.968 sec)
INFO:tensorflow:step = 10, loss = 1.5161915 (5.967 sec)
INFO:tensorflow:global_step/sec: 2.26337
INFO:tensorflow:accuracy = 0.946875 (4.418 sec)
INFO:tensorflow:step = 20, loss = 0.35078222 (4.418 sec)
INFO:tensorflow:global_step/sec: 2.25644
INFO:tensorflow:accuracy = 0.928125 (4.432 sec)
INFO:tensorflow:step = 30, loss = 0.23873076 (4.432 sec)
INFO:tensorflow:global_step/sec: 2.27079
INFO:tensorflow:accuracy = 0.953125 (4.404 sec)
INFO:tensorflow:step = 40, loss = 0.20252994 (4.404 sec)
INFO:tensorflow:global_step/s

INFO:tensorflow:step = 410, loss = 0.2221373 (4.394 sec)
INFO:tensorflow:global_step/sec: 2.27524
INFO:tensorflow:accuracy = 0.93125 (4.395 sec)
INFO:tensorflow:step = 420, loss = 0.19163156 (4.394 sec)
INFO:tensorflow:global_step/sec: 2.26619
INFO:tensorflow:accuracy = 0.95 (4.413 sec)
INFO:tensorflow:step = 430, loss = 0.19835296 (4.413 sec)
INFO:tensorflow:global_step/sec: 2.46061
INFO:tensorflow:accuracy = 0.94375 (4.064 sec)
INFO:tensorflow:step = 440, loss = 0.1853443 (4.064 sec)
INFO:tensorflow:Saving checkpoints for 451 into /tmp/tmpupper/model.ckpt.
INFO:tensorflow:global_step/sec: 1.88996
INFO:tensorflow:accuracy = 0.953125 (5.292 sec)
INFO:tensorflow:step = 450, loss = 0.15512681 (5.293 sec)
INFO:tensorflow:global_step/sec: 2.64467
INFO:tensorflow:accuracy = 0.946875 (3.780 sec)
INFO:tensorflow:step = 460, loss = 0.16933422 (3.779 sec)
INFO:tensorflow:global_step/sec: 2.65885
INFO:tensorflow:accuracy = 0.91875 (3.761 sec)
INFO:tensorflow:step = 470, loss = 0.21231374 (3.761 

INFO:tensorflow:global_step/sec: 2.6623
INFO:tensorflow:accuracy = 0.925 (3.756 sec)
INFO:tensorflow:step = 840, loss = 0.21698421 (3.756 sec)
INFO:tensorflow:global_step/sec: 2.65777
INFO:tensorflow:accuracy = 0.965625 (3.762 sec)
INFO:tensorflow:step = 850, loss = 0.14658873 (3.762 sec)
INFO:tensorflow:global_step/sec: 2.65592
INFO:tensorflow:accuracy = 0.953125 (3.765 sec)
INFO:tensorflow:step = 860, loss = 0.17321393 (3.765 sec)
INFO:tensorflow:global_step/sec: 2.65509
INFO:tensorflow:accuracy = 0.940625 (3.766 sec)
INFO:tensorflow:step = 870, loss = 0.18326941 (3.766 sec)
INFO:tensorflow:global_step/sec: 2.65435
INFO:tensorflow:accuracy = 0.94375 (3.768 sec)
INFO:tensorflow:step = 880, loss = 0.19229382 (3.768 sec)
INFO:tensorflow:global_step/sec: 2.65653
INFO:tensorflow:accuracy = 0.93125 (3.764 sec)
INFO:tensorflow:step = 890, loss = 0.1923301 (3.765 sec)
INFO:tensorflow:Saving checkpoints for 901 into /tmp/tmpupper/model.ckpt.
INFO:tensorflow:global_step/sec: 1.85564
INFO:tenso

INFO:tensorflow:accuracy = 0.940625 (3.765 sec)
INFO:tensorflow:step = 1260, loss = 0.16887942 (3.765 sec)
INFO:tensorflow:global_step/sec: 2.65737
INFO:tensorflow:accuracy = 0.9625 (3.763 sec)
INFO:tensorflow:step = 1270, loss = 0.16509286 (3.763 sec)
INFO:tensorflow:global_step/sec: 2.65953
INFO:tensorflow:accuracy = 0.94375 (3.760 sec)
INFO:tensorflow:step = 1280, loss = 0.1969862 (3.760 sec)
INFO:tensorflow:global_step/sec: 2.64995
INFO:tensorflow:accuracy = 0.928125 (3.774 sec)
INFO:tensorflow:step = 1290, loss = 0.21766907 (3.774 sec)
INFO:tensorflow:global_step/sec: 2.64849
INFO:tensorflow:accuracy = 0.925 (3.776 sec)
INFO:tensorflow:step = 1300, loss = 0.22852871 (3.777 sec)
INFO:tensorflow:global_step/sec: 2.63902
INFO:tensorflow:accuracy = 0.95 (3.789 sec)
INFO:tensorflow:step = 1310, loss = 0.14205024 (3.788 sec)
INFO:tensorflow:global_step/sec: 2.64759
INFO:tensorflow:accuracy = 0.940625 (3.777 sec)
INFO:tensorflow:step = 1320, loss = 0.15642552 (3.777 sec)
INFO:tensorflow:

INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-11-02:04:48
INFO:tensorflow:Saving dict for global step 1651: accuracy = 0.923457, global_step = 1651, loss = 0.2202893
INFO:tensorflow:Validation (step 1700): global_step = 1651, accuracy = 0.923457, loss = 0.2202893
INFO:tensorflow:Stopping. Best step: 1500 with loss = 0.2199110984802246.
INFO:tensorflow:step = 1700, loss = 0.14184755 (128.606 sec)
INFO:tensorflow:Saving checkpoints for 1701 into /tmp/tmpupper/model.ckpt.
INFO:tensorflow:Loss for final step: 0.14184755.


In [15]:
print(model.evaluate(input_fn=test_input_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-11-02:04:53
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpupper/model.ckpt-1701
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-11-02:05:41
INFO:tensorflow:Saving dict for global step 1701: accuracy = 0.90516603, global_step = 1701, loss = 0.29932573
{'global_step': 1701, 'accuracy': 0.90516603, 'loss': 0.29932573}
